<a href="https://colab.research.google.com/github/Hairun-nisha/Diabetes-Severity-Monitor/blob/main/Diabetes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -U kaleido

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.9/79.9 MB 7.7 MB/s eta 0:00:00


In [2]:
!pip install reportlab==3.6.12

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 20.7 MB/s eta 0:00:00


In [3]:
!pip install fpdf

  Preparing metadata (setup.py) ... done
  Created wheel for fpdf: filename=fpdf-1.7.2-py2.py3-none-any.whl size=40704 sha256=9e242c6007523d960adcfeb482f43a372d6d311bb096db7e6a5277969518e974
  Stored in directory: /root/.cache/pip/wheels/f9/95/ba/f418094659025eb9611f17cbcaf2334236bf39a0c3453ea455
Successfully built fpdf


In [4]:
pip install pandas scikit-learn fpdf

In [ ]:
import pandas as pd
from fpdf import FPDF
from datetime import datetime, timedelta
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, accuracy_score
import matplotlib.pyplot as plt

# Load the patient data
data = pd.read_csv('diabetes_prediction_dataset.csv')

# Features for training
X = data[['age', 'bmi', 'blood_glucose_level', 'HbA1c_level']]
y = data['diabetes']  # The target column with values indicating diabetes severity

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

# Make predictions on the test set
y_pred = model.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)

# Print classification report for accuracy and performance evaluation
print(f"Classification Report:\n{classification_report(y_test, y_pred)}")
print(f"Accuracy:{accuracy * 100:.2f}%")

# Predict categories for all rows in the dataset
data['Predicted_Category'] = model.predict(X)

# Define diet plans for Severe Diabetics based on glucose levels
diet_plans = {
    0: "Balanced diet, regular physical activity.",
    1: "High-fiber diet, regular exercise.",
    2: "Low-carb diet, blood sugar monitoring.",
    2.1: "Strict low-carb diet, glucose monitoring, insulin intake.",  # Strict diet for severe diabetics aged 50 and above
    2.2: "Low-calorie diet, increased insulin intake, blood sugar monitoring.",
    2.3: "Zero-sugar diet, medical monitoring, daily exercise."
}

# Function to calculate next checkup date based on severity
def calculate_checkup_date(category, age=None, glucose_level=None):
    today = datetime.today()
    if category == 0:  # Non-diabetic
        next_checkup = today + timedelta(weeks=26)  # 6 months
    elif category == 1:  # Moderate Diabetic
        next_checkup = today + timedelta(weeks=12)  # 3 months
    elif category == 2:  # Severe Diabetic
        if age and age >= 50:  # For patients aged 50 and above, checkup every 2 weeks
            next_checkup = today + timedelta(weeks=2)
        elif glucose_level is not None:
            if glucose_level > 200:  # More severe cases with high glucose
                next_checkup = today + timedelta(weeks=1)  # Checkup every week for higher severity
            elif glucose_level > 150:
                next_checkup = today + timedelta(weeks=2)  # Checkup every 2 weeks for moderate severity
            else:
                next_checkup = today + timedelta(weeks=4)  # Regular checkup for lower severity
        else:
            next_checkup = today + timedelta(weeks=4)  # Default for Severe Diabetic
    return next_checkup.strftime("%Y-%m-%d")

# Create a pie chart showing the percentage of diabetic and non-diabetic patients
diabetes_counts = data['Predicted_Category'].value_counts()
labels = ["Non-Diabetic", "Moderate Diabetic", "Severe Diabetic"]
sizes = [diabetes_counts.get(i, 0) for i in range(3)]
plt.figure(figsize=(6, 6))
plt.pie(sizes, labels=labels, autopct='%1.1f%%', startangle=140, colors=['#66c2a5', '#fc8d62', '#8da0cb'])
plt.title("Diabetes Prediction Distribution")
plt.savefig("diabetes_pie_chart.png")
plt.close()

# Create PDF
pdf = FPDF()
pdf.set_auto_page_break(auto=True, margin=15)
pdf.add_page()

# Title
pdf.set_font('Arial', 'B', 14)
pdf.cell(200, 10, txt="Diabetes Prediction Report", ln=True, align='C')

# Display accuracy in the PDF
pdf.set_font('Arial', '', 12)
pdf.ln(10)
pdf.cell(200, 10, txt=f"Model Accuracy: {accuracy * 100:.2f}%", ln=True)  # Show accuracy as percentage


# Function to create a table for each category
def create_table(category, category_name):
    pdf.ln(10)
    pdf.set_font('Arial', 'B', 12)
    pdf.cell(100, 5, category_name, ln=True, align='C')  # Category Name as table title

    pdf.set_font('Arial', 'B', 10)

    # Table headers with increased column sizes
    pdf.cell(50, 10, "Patient ID", border=1)
    pdf.cell(40, 10, "Age", border=1)
    pdf.cell(80, 10, "Diet Plan", border=1)
    pdf.cell(60, 10, "Checkup Date", border=1)
    pdf.ln()

    # Add rows for each patient in the category
    pdf.set_font('Arial', '', 8)
    for index, row in data[data['Predicted_Category'] == category].iterrows():
        diet_plan = diet_plans[category]

        # Adjust diet plan for severe diabetics aged 50 and above
        if category == 2:
            diet_plan = diet_plans[2.1] if row['age'] >= 50 else diet_plans[2]

        # Assign the same checkup date for all severe diabetics
        if category == 2:
            checkup_date = "2024-11-30"  # Set a fixed checkup date for all severe diabetics
        else:
            checkup_date = calculate_checkup_date(category, row['age'], row['blood_glucose_level'])

        pdf.cell(50, 10, str(index), border=1)  # Assuming Patient ID as index
        pdf.cell(40, 10, str(row['age']), border=1)
        pdf.cell(80, 10, diet_plan, border=1)
        pdf.cell(60, 10, checkup_date, border=1)
        pdf.ln()

# Create tables for each category
create_table(0, "Non-Diabetic")  # Non-Diabetic patients
create_table(1, "Moderate Diabetic")  # Moderate Diabetic patients
create_table(2, "Severe Diabetic")  # Severe Diabetic patients

# Save PDF
pdf.output("diabetes_report.pdf")

